In [3]:
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import numpy as np
import math
import os

In [4]:


def compute_DE(signal):
    variance = np.var(signal, ddof=1)
    return math.log(2 * math.pi * math.e * variance) / 2

def eliminate_channels(data):
    # Indices of rows to keep
    indices_to_keep = [1, 4, 6, 8, 18, 16, 24, 26, 36, 34, 42, 44, 46, 53, 59, 60, 61, 55, 48, 50, 40, 38, 30, 32, 22, 20, 12, 14, 5, 3, 10, 28]

    # Create a new array to store the selected rows
    new_data = []

    # Append selected rows to new_data
    for i in range(len(indices_to_keep)):
        new_data.append(data[indices_to_keep[i]-1])

    # Convert new_data to a NumPy array
    return np.array(new_data)

def load_data():
    data_dir = "E:/STUDY/Publications/Thesis/Brain Emotion Detection/Dataset/SEED/SEED_EEG/Preprocessed_EEG/"
    fs = 200
    fStart = [4, 8, 14, 31]
    fEnd = [7, 13, 30, 50]
    all_channels = range(62)
    selected_channels = range(32)
    seconds_to_exclude = 50

    filename_label = "label"
    label = loadmat(data_dir + filename_label)
    label = label["label"][0]

    datasets_X, datasets_y = [], []

    for filename_data in os.listdir(data_dir):
        if filename_data in ["label.mat", "readme.txt"]:
            continue

        data_all = loadmat(data_dir + filename_data)
        scenes = list(data_all.keys())[3:]

        for index, scene in enumerate(scenes):
            dataset_X = []
            data = data_all[scene][:, :37000]  # Considering all channels
            
            new_data = eliminate_channels(data)
            data = np.array(new_data)

            start_index = seconds_to_exclude * fs
            

            data = data[:, start_index:]

            for band_index, band in enumerate(fStart):
                b, a = signal.butter(4, [fStart[band_index]/fs, fEnd[band_index]/fs], 'bandpass')
                filtedData = signal.filtfilt(b, a, data)
                filtedData_de = []

                for lead in selected_channels:
                    filtedData_split = []

                    for de_index in range(0, filtedData.shape[1] - int(fs*3.2), int(fs*3.2)):
                        filtedData_split.append(compute_DE(filtedData[lead, de_index: de_index + int(fs*3.2)]))

                    filtedData_de.append(filtedData_split)
                
                filtedData_de = np.array(filtedData_de)
                dataset_X.append(filtedData_de)

            datasets_X.append(dataset_X)
            datasets_y.append(label[index])

    datasets_X, datasets_y = np.array(datasets_X), np.array(datasets_y)

    return datasets_X, datasets_y


datasets_X, datasets_y = load_data()
print(datasets_X.shape)
print(datasets_y.shape)


(675, 4, 32, 42)
(675,)


In [5]:
new_datasets_X = np.copy(datasets_X)
new_datasets_X = np.transpose(new_datasets_X, (0, 2, 3, 1))
print(new_datasets_X.shape)

(675, 32, 42, 4)


In [6]:
np.save('SEED_DE_X_for_Merge_2min.npy', new_datasets_X)
np.save('SEED_DE_y_for_Merge_2min.npy', datasets_y)